In [ ]:
# | default_exp _components.integration_test_generator

In [ ]:
# | export


from typing import *
import shutil
from pathlib import Path
import os
from tempfile import TemporaryDirectory
from contextlib import contextmanager
import subprocess # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    download_and_extract_faststream_archive,
    write_file_contents,
    read_file_contents,
)

In [ ]:


import pytest

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

@contextmanager
def set_cwd(cwd_path: Union[Path, str]) -> Generator:
    """Set the current working directory for the duration of the context manager.

    Args:
        cwd_path: The path to the new working directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)

    try:
        yield
    finally:
        os.chdir(original_cwd)

In [ ]:
# | export


def _setup_venv_and_run_tests(
    env_name: str = "venv", file_to_test: Optional[str] = None
) -> bool:
    try:
        # Create virtual environment
        subprocess.run(["python3", "-m", "venv", env_name], check=True) # nosec: B602, B603, B607 subprocess call - check for execution of untrusted input.

        # Install requirements and run pytest
        commands = [
            "venv/bin/pip install --upgrade pip",
            "venv/bin/pip install -q -r dev_requirements.txt",
            "venv/bin/python -m pytest"
            if not file_to_test
            else f"venv/bin/python -m pytest {file_to_test}",
        ]

        for command in commands:
            # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
            subprocess.run( # nosec: B602, B603 subprocess call - check for execution of untrusted input.
                command,
                check=True,
                shell=True,
                executable="/bin/bash",
                stderr=subprocess.DEVNULL,
                stdout=subprocess.PIPE,
            )
        return True
    except Exception as e:
        return False

In [ ]:
fixture_dev_requirements = """pytest
"""
fixture_app = """
print('hi')
"""

fixture_test = """
import pytest

def test_always_fails():
    assert True
"""

with TemporaryDirectory() as d:
    req_file = Path(d) / "dev_requirements.txt"
    write_file_contents(req_file, fixture_dev_requirements)
    
    app_file = Path(d) / "application.py"
    write_file_contents(app_file, fixture_app)
    
    test_file = Path(d) / "test.py"
    write_file_contents(test_file, fixture_test)
    
    with set_cwd(d):
        actual = _setup_venv_and_run_tests(file_to_test=test_file)
        print(actual)
        assert actual

True


In [ ]:
fixture_dev_requirements = """pytest
"""
fixture_app = """import pytest

def test_always_fails():
    assert False
"""

with TemporaryDirectory() as d:
    req_file = Path(d) / "dev_requirements.txt"
    write_file_contents(req_file, fixture_dev_requirements)
    
    app_file = Path(d) / "application.py"
    write_file_contents(app_file, fixture_app)
    
    with set_cwd(d):
        actual = _setup_venv_and_run_tests(file_to_test=app_file)
        print(actual)
        assert not actual

False


In [ ]:
# | export


def run_integration_test(src_dir: str) -> None:
    with yaspin(
        text="Running integration tests...", color="cyan", spinner="clock"
    ) as sp:
        with TemporaryDirectory() as d:
            shutil.copytree(src_dir, d, dirs_exist_ok=True)
            with set_cwd(d):
                status_ok = _setup_venv_and_run_tests()
            sp.text = ""
            if status_ok:
                message = " ✔ Integration tests were successfully completed."
            else:
                message = " ✘ Error: Integration tests failed."
                sp.color = "red"
            sp.ok(message)

In [ ]:
fixture_dev_requirements = """pytest
"""
fixture_app = """import pytest

def test_always_fails():
    assert False
"""

with TemporaryDirectory() as d:
    req_file = Path(d) / "dev_requirements.txt"
    write_file_contents(req_file, fixture_dev_requirements)
    
    app_file = Path(d) / "application.py"
    write_file_contents(app_file, fixture_app)

    run_integration_test(d)

⠹ Running integration tests... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✘ Error: Integration tests failed. 


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
